In [ ]:
#Text mining Properati Dataset (www.properati.com.ar)

In [98]:
#Instalación de paquetes
list.of.packages <- c('tm', 'wordcloud', 'data.table')

new.packages <- list.of.packages[!(list.of.packages %in% 
                                   installed.packages()[,"Package"])]

if(length(new.packages)) install.packages(new.packages, 
                                          repos = "https://cran.r-project.org")

library(tm)
library(wordcloud)
library(data.table)    
    
#Configuraciones adicionales
options(warn=-1, scipen = 999, repr.plot.width=7, repr.plot.height=5)

In [99]:
#Carga de datos
#Carga de datos
df <- fread(input = 'data/properati-AR-2016-12-01-properties-rent.csv', 
                          encoding = 'UTF-8')
df <- as.data.frame(df)

In [100]:
#Utilizamos solo la locación del inmueble, el titulo del aviso y la descripción
properati <- df[, c('place_name', 'title', 'description')]

In [104]:
#Exploramos la estructura de datos
str(properati)
table(Encoding(properati$place_name))
table(Encoding(properati$title))
table(Encoding(properati$description))

tail(properati$description[Encoding(properati$description) =='UTF-8'],1)
head(properati$description[Encoding(properati$description) =='unknown'],501)

'data.frame':	42091 obs. of  3 variables:
 $ place_name : chr  "Quilmes" "Tigre" "Quilmes" "Tigre" ...
 $ title      : chr  "Imperdible" "ph 3 amb talar a 2 de colectora" "Precio Increible!!!!!" "ALQUILER" ...
 $ description: chr  ".Publicado por EDUARDO RODRIGUEZ PROPIEDADES a traves INMOMAP" ".Publicado por Herrera &amp; Cia a traves INMOMAP" ".Publicado por Servicios Profesionales Alvarez Casal a traves INMOMAP" "CASA CENTRO DE BENAVIDEZ CON GAS NATURAL SIN COCHERA CON PIELTA .Publicado por piperse a traves INMOMAP" ...



unknown   UTF-8 
  38475    3616 


unknown   UTF-8 
  37841    4250 


unknown   UTF-8 
  22858   19233 

[1] "Muy linda vista y mucha luz. Reciclado.<br>Living comedor, toilette, 3 dormitorios, baÃ±o completo, cocina, lavadero.<br>El edificio tiene SUM, patio de uso comÃºn<br><br>[ Dixon45400911 ]"

[1] ".Publicado por EDUARDO RODRIGUEZ PROPIEDADES a traves INMOMAP"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
  [2] ".Publicado por Herrera &amp; Cia a traves INMOMAP"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [79]:
#Existe un problema de codificación de caracteres en las columnas por lo tanto

In [33]:
#Armamos una columna que contenga el titulo y la descripción
properati$text <- paste0(properati$title, properati$description)
properati[, c('title', 'description')] <- NULL
str(properati)

'data.frame':	22297 obs. of  2 variables:
 $ place_name: chr  "Quilmes" "Tigre" "Quilmes" "Tigre" ...
 $ text      : chr  "Imperdible.Publicado por EDUARDO RODRIGUEZ PROPIEDADES a traves INMOMAP" "ph 3 amb talar a 2 de colectora.Publicado por Herrera &amp; Cia a traves INMOMAP" "Precio Increible!!!!!.Publicado por Servicios Profesionales Alvarez Casal a traves INMOMAP" "ALQUILERCASA CENTRO DE BENAVIDEZ CON GAS NATURAL SIN COCHERA CON PIELTA .Publicado por piperse a traves INMOMAP" ...


In [34]:
#Chequeamos algunas observaciones
tail(properati)

,place_name,text
22292,Vicente López,"Departamento con Plantas en Miguel De Azcuénaga 1200, Bs.As. G.B.A. Zona Norte, Vicente López, por $ 16.500Muy buen departamento<br><br>Hall, living comedor, estar Ã­ntimo, cocina, dep. servicio, 3 dormitorios, 2 baÃ±os.<br>Calefactores en dormitorios y living-comedor. Agua caliente por 2 calefones.<br>OrientaciÃ³n: frente nor-este. BalcÃ³n con vista al rÃ­o.Cochera. Baulera.<br><br>Expensas $ 3105<br> A.B.L. $ 357<br><br>[ Dixon9801043 ]"
22293,Palermo,"Departamento 49m² con Cochera en Arevalo 3000, Capital Federal, Palermo, por $ 14.000Excelente y moderno departamento de 2 ambientes al frente, tercer piso, muy luminoso, en el Complejo Terrazas al Polo, la mejor zona de CaÃ±itas. <br>EdificaciÃ³n nueva (antigÃ¼edad 3 aÃ±os) de 4 pisos con detalles de categorÃ­a del Estudio Gradel Kopellioff; terminaciones de calidad. <br>Dormitorio con placard espejado. <br>Cocina completa, con mesada de marmol. ConexiÃ³n para lavarropas o lavavajillas. <br>BaÃ±o completo <br>CalefacciÃ³n por losa radiante. Pisos flotantes. <br>Seguridad 24 hs y estacionamiento de cortesÃ­a. <br>Amenities: Gimnasio, Sum, Sala de masajes, Sauna, Laundry, Parrillas, Yacuzzi, Solarium y gran pileta climatizada de 22 metros. <br>Expensas: 3800$<br>Las medidas son aproximadas y al solo efecto orientativo. Las medidas reales surgen del titulo de propiedad.<br><br>[ Dixon72700150 ]"
22294,Palermo,"Departamento 87m² con Cochera en Arevalo 3000, Capital Federal, Palermo, por $ 21.000Excelente y moderno departamento, de 3 ambientes al frente, tercer piso, muy luminoso, en el Complejo Terrazas al Polo, la mejor zona de CaÃ±itas. <br>EdificaciÃ³n nueva (antigÃ¼edad 4 aÃ±os) de 4 pisos con detalles de categorÃ­a del Estudio Gradel Kopellioff; terminaciones de calidad. <br>Dos dormitorios con placards. <br>Cocina completa, con mesada de marmol. ConexiÃ³n para lavarropas o lavavajillas. <br>Dos BaÃ±os. <br>CalefacciÃ³n por losa radiante. Pisos flotantes. <br>Seguridad 24 hs y estacionamiento de cortesÃ­a. <br>Amenities: Gimnasio, Sum, Sala de masajes, Sauna, Laundry, Parrillas, Yacuzzi, Solarium y gran pileta climatizada de 22 metros. <br>Expensas: 6800$<br>Las medidas son aproximadas y al solo efecto orientativo. Las medidas reales surgen del titulo de propiedad.<br><br>[ Dixon72700152 ]"
22295,Bs.As. G.B.A. Zona Norte,"Departamento 76m² con Teléfono, Argentina, Bs.As. G.B.A. Zona Norte, por $ 13.000Libre 1 de Enero 2017<br>Muy buen depto. en 3Â° piso en Jardines de San Isidro II.<br><br>Living-comedor con salida al balcÃ³n. Cocina con lavadero incorporado.<br>Dos dormitorios, uno en suite, un baÃ±o completo mÃ¡s. Techos altos!<br>Mira hacia la pileta. Cochera doble con baulera.<br><br>Expensas: $ 4.800 (incluyen AYSA) ABL: $ 770<br><br>El Condominio cuenta con seguridad las 24 hs, pileta para niÃ±os, laundry, sauna, etc.<br><br>[ Dixon1102239 ]"
22296,San Isidro,"Local con Teléfono en Juan José Díaz 500, Bs.As. G.B.A. Zona Norte, San Isidro, por $ 10.000Necesita algunas reparaciones<br>Posibilidad de Armar un Local en San Isidro Centro a una cuadra de Tribunales San Isidro<br>Desarrollado en 2 Plantas de unos 15 m2 cubiertos cada una. Un baÃ±o en planta baja<br>Actualmente se accede a la planta alta por una escalera externa<br><br>[ Dixon10204661 ]"
22297,Belgrano,"Departamento 90m² con Cochera en Juramento 3100, Capital Federal, Belgrano, por $ 14.000Muy linda vista y mucha luz. Reciclado.<br>Living comedor, toilette, 3 dormitorios, baÃ±o completo, cocina, lavadero.<br>El edificio tiene SUM, patio de uso comÃºn<br><br>[ Dixon45400911 ]"


In [35]:
#Agrupamos todas las cadenas de texto por la ubicación del inmueble
properati <- aggregate(text ~ place_name, data = properati, paste0)

In [39]:
tail(properati,2)

place_name
563 Zárate    
564 Zelaya    
    text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [40]:
#Se puede ver que existen caracteres especiales en donde deberían haber caracteres con tildes: ej. hectÃ¡rea
#Los datos tambien acarrean ciertas etiquetas html que debemos eliminarlos.

#Quitar las etiquetas html
cleanFun <- function(htmlString) {
  return(gsub("<.*?>", "", htmlString))
}

#Transformación caracteres no reconocidos
properati$text <- cleanFun(properati$text)
properati$text <- iconv(properati$text, from  = 'UTF-8')
properati$place_name <- iconv(properati$place_name, from = 'UTF-8')
tail(properati,2)

,place_name,text
563,Zárate,"c(""ALQUILA LOCALES EN VILLA MASSONICODIGO: ubicado en: PUEYRREDON 940 - Publicado por: PAULA GUERRERO - NEGOCIOS INMOBILIARIOS. El precio es de ARS 4000. EXCELENTE LOCAL CON BAÑO- . Publicado a través de Mapaprop"", ""DEPARTAMENTO DE CATEGORIA CENTRICO CON COCHERACODIGO: 2593-ALQUI78 ubicado en: HIPOLITO YRIGOYEN Y BROWN - Publicado por: PAULA GUERRERO - NEGOCIOS INMOBILIARIOS. El precio es de ARS 12000. dos dormitorios, dos baños, amplio living-comedor, cocina y cochera . Publicado a través de Mapaprop"", ""Casa 3 dormitorios, ESTEÑA, ALQ.TEMPORADACorredor Responsable: Carlos Belsito - C.M.C.P.S.I 5790Contacto: Lilian García - MLS ID # 420241053-192La mejor opción en la mejor ubicación de Chacras, excelentes vistas, campo y río. Confort y glamour. El Club de Chacras del Parana ofrece variedad de posibilidades para su disfrute, magnífica casa, en Chacras del Parana, a solo 103 Km de Capital, con galerías, parrilla y piscina con cerco, estar comedor con hogar y completamente equipada, de estilo esteño, muy relajada.Chacras del Parana se encuentra sobre el Río Paraná de las Palmas, próximo a la confluencia de los ríos Areco y Baradero, su paisaje es sorprendentemente ondulado y atractivo. La casa en alquiler a pasos de la costa del río y consta de tres dormitorios, estar comedor con hogar a leña y parrilla y sector de bar y parrillero. todo muy funcional. El estado de la propiedad es bueno, el equipamiento muy completo. Se accede por Panamericana, ramal Campana, en el km.103 se toma a la derecha por pavimento hasta el crece de vías del F.C. Belgrano Cargas, a mano izquierda por mejorado, en 5' minutos se encuentra el acceso. A partir de allí todo es soñado. Chacras cuenta con Club House, Restaurante con delivery, Monturero, Amarras y un espectacular golf de 9 hoyos, así como caminos con hermosas vistas para aquellos que gusten de realizar caminatas ó deportes al aire libre. Enero 2017 $ 58.000 y FEBRERO 2017 48.000 se alquila también quincenal. FEBRERO ALQUILADA. "", ""Casa 3 dormitorios, tipo esteñaCorredor Responsable: Carlos Belsito - C.M.C.P.S.I 5790Contacto: Lilian García - MLS ID # 420241053-250La casa en alquiler està sobre la barranca, vistas largas, a pasos de la costa del río y consta de tres dormitorios, estar comedor con hogar a leña y parrilla y sector de bar y parrillero, todo muy funcional. El estado de la propiedad es bueno, el equipamiento muy completo. La mejor opción en la mejor ubicación de Chacras, excelentes vistas, campo y río. Confort y glamour. El Club de Chacras del Parana ofrece variedad de posibilidades para su disfrute, magnífica casa, en Chacras del Parana, a solo 103 Km de Capital, con galerías, parrilla y piscina con cerco, estar comedor con hogar y completamente equipada, de estilo esteño, muy relajada.Chacras del Parana se encuentra sobre el Río Paraná de las Palmas, próximo a la confluencia de los ríos Areco y Baradero, su paisaje es sorprendentemente ondulado y atractivo. Se accede por Panamericana, ramal Campana, en el km.103 se toma a la derecha por pavimento hasta el crece de vías del F.C. Belgrano Cargas, a mano izquierda por mejorado, en 5' minutos se encuentra el acceso. A partir de allí todo es soñado. Chacras cuenta con Club House, Restaurante con delivery, Monturero, Amarras y un espectacular golf de 9 hoyos, así como caminos con hermosas vistas para aquellos que gusten de realizar caminatas ó deportes al aire libre. Enero $ 52.000 y "", ""MONOAMBIENTE SOBRE AVENIDA LAVALLE ( FRENTE AL BINGO) CODIGO: 2593-ALQU99 ubicado en: LAVALLE 1405 - Publicado por: PAULA GUERRERO - NEGOCIOS INMOBILIARIOS. El precio es de ARS 4500. LOCAL CON BAÑO Y KITCHINET opcion amoblado . Publicado a través de Mapaprop"")"
564,Zelaya,"c(""Departamento 5000m² con Cochera, Pilar, Zelaya, por U$S 3.500QUINTA ESPECTACULAR EN EL MEDIO DE ESTANCIAS DE PILAR , AL LADO DE LA ASOCIACION DE POLO LA PACIENCIA: SON 2 CASAS CASA CHICA DE 120 M2, 4 CUARTOS Y 2 BAÃOS MAS LIVING Y COCINA CASA GRA

In [41]:
#Volvemos a chequear y vemos que eliminamos las etiquetas html
str(properati)

'data.frame':	564 obs. of  2 variables:
 $ place_name: chr  "9 de Abril" "Abasto" "Abril Club de Campo" "Acacias Blancas" ...
 $ text      : chr  "Casa Quinta en Alquiler 9 De Abril / E Echeverria (A170 1269)Hermosa casa quinta de 4 ambientes amoblada, la cual consta de: am"| __truncated__ "c(\"22\", \"Depto. 2 Ambientes en alquiler.Excelente Depto. de 2 ambientes recién pintado. Cocina independiente con lavadero in"| __truncated__ NA "c(\"Casa en Alquiler Anual en Acacias BlancasPropiedad desarrollada en una planta con hall de recepcion - living-comedor- cocin"| __truncated__ ...


In [12]:
myReader <- readTabular(mapping=list(content="text", id="place_name"))
corpus_clean <- VCorpus(DataframeSource(properati), readerControl=list(reader=myReader))

In [13]:
for (i in 1:length(corpus_clean)) {
  attr(corpus_clean[[i]], "place_name") <- properati$place_name[i]
}

In [14]:
#Stop words en castellano
es_words <- readLines("data/words.txt")

#Tambien quito las palabras que esten en el nombre del lugar porque reaparecen varias veces en 
#el titulo o la descripcion por lo que resultará en una palabra frecuente y no queremos eso
split_words <- function(x){ strsplit(x, "[^[:alpha:]]+") }
places_words <- tolower(unlist(lapply(properati$place_name, split_words)))

In [15]:
#Limpiar texto
skip_es_words <- function(x) removeWords(x, es_words)
skip_place_words <- function(x) removeWords(x, places_words)
funcs <- list(content_transformer(tolower), removePunctuation, removeNumbers, stripWhitespace, skip_es_words, skip_place_words)
corpus_clean <- tm_map(corpus_clean, FUN = tm_reduce, tmFuns = funcs)
corpus_clean <- tm_map(corpus_clean, removeWords, stopwords('es'))
corpus_clean <- tm_map(corpus_clean, removeWords, c("alquilo", "alquiler", "alq", 'depto', 'dpto', 'casa', 'departamento', 'ambientes', 'local', 'ambiente', 'excelente', 'apto'))


In [ ]:
#Armo el DTM
dtm <- DocumentTermMatrix(corpus_clean, control = list(wordLengths = c(4,17))) #limito a palabras entre 4 y 17 caracteres

rowTotals <- apply(dtm , 1, sum)
dtm <- dtm[rowTotals > 0, ] 

colTotals <- apply(dtm, 2, sum)
dtm <- dtm[,colTotals > 20]

tst <- apply(dtm, 1, function(x) unlist(dtm[["dimnames"]][2], use.names = FALSE)[x == max(x)])


In [ ]:
#Buscar lugars con palabra mas frecuente "Subte"
idx <- grep('subte', tst)
tst[idx] 

In [48]:
#Mostrar todos los lugares y sus palabras más frecuentes

$`9 de Abril`
[1] "amoblada" "quinta"  

$Abasto
[1] "cocina"

$`Abril Club de Campo`
[1] "baño"

$`Acacias Blancas`
[1] "baño"

$Acassuso
[1] "acassuso"

$Adrogué
[1] "baño"

$`Aeropuerto Internacional Ezeiza`
[1] "pisos"

$Agronomía
[1] "expensas"

$`Albanueva Barrio Cerrado`
[1] "albanueva"

$`Aldea Protestante`
[1] "amplio"     "comedor"    "parquizado" "terreno"   

$`Aldo Bonzi`
[1] "publicado"

$Alfar
[1] "camas"

$Almagro
[1] "cuadras"

$`Almirante Brown`
[1] "comercial" "publicado"

$`Alta Gracia`
[1] "dormitorios"

$Altamira
[1] "negocios"  "oficina"   "propiedad" "venta"    

$`Altos de Hudson I`
[1] "altos"

$`Altos de Hudson II`
 [1] "alquilerventa" "altos"         "baño"          "flotante"     
 [5] "fríocalor"     "goldoni"       "jacuzzi"       "parrilla"     
 [9] "pisos"         "porcelanato"   "propiedades"   "recepción"    
[13] "suite"        

$`Altos de Manzanares 1 y 2`
[1] "roca"

$Anisacate
[1] "baño"        "cama"        "cocina"      "dormitorios" "propiedad"  
[6] "ropero"     

$Añelo
[1] "dormitorios"   "frente"        "habitaciones"  "hotel"        
[5] "inmobiliarias" "inversiones"   "ruta"          "solar"        
[9] "urbano"       

$Apóstoles
[1] "publicado"

$Ascochinga
[1] "asador"

$Avellaneda
[1] "cocina" "pisos"  "precio"

$Azul
[1] "publicado"

$`Bahía Blanca`
[1] "baño"

$Balcarce
[1] "dormitorios"

$Balvanera
[1] "propiedades"

$Banfield
[1] "baño"

$Baradero
[1] "publicado"

$`Barbarita, Barrio Cerrado`
[1] "baño"         "comedor"      "habitaciones"

$Barracas
[1] "comercial"

$`Barrancas de Santa María`
[1] "lago"

$`Barrio Alamo Alto`
[1] "madera"     "mooswalder" "negocios"  

$`Barrio Cabos del Lago`
[1] "área" "lago"

$`Barrio Cerrado`
[1] "duplex"

$`Barrio Cerrado ""Soles de Pilar""`
[1] "pilar"

$`Barrio Cerrado El Lauquén`
[1] "amueblada"  "familias"   "lauquen"    "permanente"

$`Barrio Cerrado Los Troncos`
[1] "publicado"

$`Barrio El Golf`
[1] "lago"

$`Barrio El Yacht`
[1] "cmcpsi"  "comedor"

$`Barrio La Alameda`
[1] "dormitorios"

$`Barrio La Isla`
[1] "planta" "salida"

$`Barrio Lagos del Golf 2`
[1] "aires"    "complejo" "mansa"    "vidrio"   "vista"   

$`Barrio Las Glorietas`
[1] "baño"

$`Barrio Los Alisos`
[1] "baño"        "comedor"     "dormitorios"

$`Barrio Los Castores`
[1] "baño"

$`Barrio Los Jazmines - Pilar del Este`
[1] "comedor"

$`Barrio Los Sauces`
[1] "comedor"     "dormitorios" "sauces"     

$`Barrio Norte`
[1] "cocina"

$`Barrio Parque General San Martin`
[1] "propiedades"

$`Barrio Privado ""Ayres de Pilar""`
[1] "acceso" "baño"  

$`Barrio Privado Altos del Sol`
[1] "placard" "planta" 

$`Barrio Privado El Rodal`
[1] "barrio"

$`Barrio Privado María Eugenia`
[1] "baño"     "central"  "completo" "piso"    

$`Barrio Privado Villa Olivos`
[1] "barrio"      "dormitorios" "niños"      

$`Barrio San Agustin`
[1] "amplio"      "aprox"       "baño"        "completo"    "condiciones"
[6] "jardín"      "planta"     

$`Barrio San Francisco`
 [1] "baño"        "cocina"      "comedor"     "compartido"  "construida" 
 [6] "dormitorios" "imperdible"  "jardín"      "linda"       "lindo"      
[11] "living"      "pileta"      "planta"      "principal"   "suite"      
[16] "toilette"   

$`Barrio San Isidro Labrador`
[1] "temporada"

$`Barrio San Juan`
[1] "calle"  "vereda"

$`Barrio Santa Clara`
 [1] "aparte"      "cochera"     "comedor"     "completo"    "cubierta"   
 [6] "dormitorios" "expensas"    "gran"        "incluyen"    "inmejorable"
[11] "laguna"      "living"      "nueva"       "pileta"      "piletero"   
[16] "santa"       "separado"    "tamaño"      "temporario"  "toilette"   
[21] "valores"     "vista"       "yacuzzi"    

$`Barrio Santa Teresa`
[1] "barra"      "comedor"    "protección"

$BarrioPortezuelo
[1] "vista"

$Beccar
[1] "beccar"

$`Belén de Escobar`
[1] "cocina"  "escobar"

$Belgrano
[1] "cocina"

$`Bell Ville`
[1] "propiedades" "tucumán"    

$`Bella Vista`
[1] "publicado"

$Benavidez
[1] "comedor"

$Berazategui
[1] "baño"

$`B